<a href="https://colab.research.google.com/github/thomasamged/translation/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Dense,LSTM,Dropout,Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

BankNote


In [ ]:
df=pd.read_csv("/content/eng_-french.csv")
df


,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !
...,...,...
175616,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
175617,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
175618,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
175619,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...


In [ ]:
df.isnull().sum()

,0
English words/sentences,0
French words/sentences,0


In [ ]:
df.duplicated().sum()

0

In [ ]:
en=[str(sen)for sen in df['English words/sentences']]
fr=[str(sen)for sen in df['French words/sentences']]

In [ ]:
tokn_en=Tokenizer()
tokn_en.fit_on_texts(en)
tokn_fr=Tokenizer()
tokn_fr.fit_on_texts(fr)
en_seq=tokn_en.texts_to_sequences(en)
fr_seq=tokn_fr.texts_to_sequences(fr)
max_len=max([len(sen) for sen in en_seq+fr_seq])

In [ ]:
pad_en=pad_sequences(en_seq,maxlen=max_len,padding='post')
pad_fr=pad_sequences(fr_seq,maxlen=max_len,padding='post')

In [ ]:
en_size=len(tokn_en.word_index)+1
fr_size=len(tokn_fr.word_index)+1

In [ ]:
pad_en

array([[2818,    0,    0, ...,    0,    0,    0],
       [ 429,    0,    0, ...,    0,    0,    0],
       [ 429,    0,    0, ...,    0,    0,    0],
       ...,
       [ 621,    6,   97, ...,    0,    0,    0],
       [ 370,   53,   21, ...,    0,    0,    0],
       [  62,  289,   79, ...,    0,    0,    0]], dtype=int32)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(pad_en,pad_fr,test_size=.3,random_state=1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=.2,random_state=1)

In [ ]:
model=Sequential()
model.add(Embedding(en_size,100,input_length=max_len))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(256,return_sequences=True
               ))
model.add(Dropout(.2))
model.add(Dense(fr_size,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=10,batch_size=64,validation_data=(X_val,y_val))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1537/1537 ━━━━━━━━━━━━━━━━━━━━ 262s 168ms/step - accuracy: 0.8789 - loss: 1.3638 - val_accuracy: 0.8946 - val_loss: 0.7323
Epoch 2/10
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 264s 170ms/step - accuracy: 0.8969 - loss: 0.7035 - val_accuracy: 0.9028 - val_loss: 0.6400
Epoch 3/10
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 262s 170ms/step - accuracy: 0.9038 - loss: 0.6135 - val_accuracy: 0.9074 - val_loss: 0.5803
Epoch 4/10
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 263s 170ms/step - accuracy: 0.9082 - loss: 0.5478 - val_accuracy: 0.9104 - val_loss: 0.5430
Epoch 5/10
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 262s 170ms/step - accuracy: 0.9115 - loss: 0.4995 - val_accuracy: 0.9130 - val_loss: 0.5163
Epoch 6/10
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 262s 170ms/step - accuracy: 0.9140 - loss: 0.4638 - val_accuracy: 0.9146 - val_loss: 0.4972
Epoch 7/10
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 263s 171ms/step - accuracy: 0.9161 - loss: 0.4342 - val_accuracy: 0.9156 - val_loss: 0.4858
Epoch 8/10
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 323s 171ms/step - accuracy: 0.9

In [ ]:
model.evaluate(X_test,y_test)

1647/1647 ━━━━━━━━━━━━━━━━━━━━ 91s 55ms/step - accuracy: 0.9185 - loss: 0.4638


[0.46287375688552856, 0.9186636209487915]

In [ ]:
phase="i'm happy to see you too"
phase=tokn_en.texts_to_sequences([phase])
phase=pad_sequences(phase,maxlen=max_len,padding='post')
phase

array([[ 22, 145,   3,  76,   2,  99,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]], dtype=int32)

In [ ]:
pre=model.predict(phase)
pre

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step


array([[[4.25896607e-04, 8.99390817e-01, 1.27164094e-04, ...,
         4.82203122e-09, 4.43371340e-09, 4.48815873e-09],
        [2.73346144e-04, 5.81465778e-04, 2.12186738e-03, ...,
         3.70413457e-11, 4.32359773e-11, 4.11662163e-11],
        [2.24916352e-04, 1.00389436e-04, 3.42473923e-03, ...,
         8.91421548e-10, 9.03375541e-10, 8.55801430e-10],
        ...,
        [9.99997616e-01, 5.86757203e-08, 2.19777363e-07, ...,
         3.22315756e-16, 4.05996153e-16, 4.30484299e-16],
        [9.99997497e-01, 5.84009321e-08, 2.20102024e-07, ...,
         3.22551840e-16, 4.05924923e-16, 4.30620512e-16],
        [9.99997497e-01, 5.81624882e-08, 2.20454950e-07, ...,
         3.22852219e-16, 4.05962086e-16, 4.30860435e-16]]], dtype=float32)

In [ ]:
pre=np.argmax(pre,axis=-1)
pre

array([[  1,  22, 223,   2,   4, 114,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]])

In [ ]:
trans=[]
for i in pre[0]:
  if i in tokn_fr.index_word:
    trans.append(tokn_fr.index_word[i])
  else:
    trans.append(' ')
trans=' '.join(trans)
trans

'je suis heureux de vous voir                                                                                                  '